In [57]:
#import libraries

from bs4 import BeautifulSoup
import requests
import time
import datetime

import smtplib
import re

import datetime

today = datetime.date.today()
print(today)

2024-08-12


In [58]:
#Connect to Website

URL = 'https://www.amazon.sg/s?k=sunscreen&s=exact-aware-popularity-rank&crid=26XGF0ETVPQUI&qid=1723367023&sprefix=sun%2Caps%2C833&ref=sr_st_exact-aware-popularity-rank&ds=v1%3A3pDT%2BAG86jcKv9fBxlUHlsFORcPucnxPmCLjzaZBAa4'
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36"}

page = requests.get(URL, headers=headers)

soup = BeautifulSoup(page.content, "html.parser")
soup2 = BeautifulSoup(soup1.prettify(), "html.parser")

In [59]:
# functions

def get_title(soup):
    try:
        product_title = soup.find("span", attrs={"id":'productTitle'})
        title_value = product_title.text
        title_string = title_value.strip()
    
    except AttributeError:
        title_string = ""
    
    return title_string
    

def get_price(soup):
    price = soup.find(class_='a-offscreen')
    if price:
        price_text = price.text.strip()
        try:
            price = float(price_text[2:])
            return price
        except ValueError:
            return "N/A"
    else:
        return "N/A"

def get_rating(soup):
    rating = soup.find(class_='a-icon-alt')
    if rating:
        ratings = rating.text.split(' out of 5 stars')
        ratings = ratings[0]
        rat = ratings.strip()
        return rat
    else:
        return "N/A"
    
def get_reviews(soup):
    reviews_element = soup.find("span", attrs={'id':'acrCustomerReviewText'})
    if reviews_element:
        reviews = reviews_element.text.split(' ratings')
        reviews = reviews[0]
        rev = reviews.strip()
        return rev
    else:
        return "N/A"

def get_sales(soup):
    sales = soup.find(class_='a-size-small social-proofing-faceout-title-text')
    if sales:
        return sales.text
    else:
        return "N/A"

In [ ]:
#Fetching Pages
pages = soup.find_all('a', class_='s-pagination-item s-pagination-button')

pages_list = ['https://www.amazon.sg/s?k=sunscreen&s=exact-aware-popularity-rank&crid=26XGF0ETVPQUI&qid=1723484528&sprefix=sun%2Caps%2C833&ref=sr_pg_1']

for page in pages:
    page_url = 'https://www.amazon.sg' + page.get('href')
    pages_list.append(page_url)
    new_page = requests.get(page_url, headers=headers)
    new_page_soup = BeautifulSoup(new_page.content, "html.parser")
    
# Extract the URLs from the current page
    new_pages = new_page_soup.find_all('a', class_='s-pagination-item s-pagination-button')

    # Iterate through the new pages and add them to the list if they're not already in it
    for new_page in new_pages:
        new_url = 'https://www.amazon.sg' + new_page.get('href')
        if new_url not in pages_list:
            pages_list.append(new_url)
 
    #print(pages_list)

# Create a dictionary to store unique URLs based on last 4 characters
unique_urls = {}
for url in pages_list:
    last_four = url[-4:]
    if last_four not in unique_urls:
        unique_urls[last_four] = url
        
# Convert the dictionary back to a list
page_link_list = list(unique_urls.values())

print(page_link_list)

In [168]:
for unique_url in page_link_list:
    links = soup.find_all('a', attrs={'class': 'a-link-normal s-no-outline'})

    links_list = []

    for link in links:
        links_list.append(link.get('href'))

    d = {'title': [], 'price': [], 'rating': [], 'reviews': [], 'monthly_sales': [], 'URL': []}

# Looping and extracting product details from each link
    for one_link in links_list:
        new_product = requests.get("https://www.amazon.sg" + one_link, headers=headers)

        product_soup = BeautifulSoup(new_product.content, "html.parser")

        d['title'].append(get_title(product_soup))
        d['price'].append(get_price(product_soup))
        d['rating'].append(get_rating(product_soup))
        d['reviews'].append(get_reviews(product_soup))
        d['monthly_sales'].append(get_sales(product_soup))
        d['URL'].append("https://www.amazon.sg" + one_link)

import pandas as pd
amazon_df = pd.DataFrame.from_dict(d)

amazon_df

NameError: name 'page_link_list' is not defined

In [61]:
# Fetching Links
links = soup.find_all('a', attrs={'class': 'a-link-normal s-no-outline'})

links_list = []

for link in links:
    links_list.append(link.get('href'))

d = {'title': [], 'price': [], 'rating': [], 'reviews': [], 'monthly_sales': [], 'URL': []}

# Looping and extracting product details from each link
for one_link in links_list:
    new_product = requests.get("https://www.amazon.sg" + one_link, headers=headers)

    product_soup = BeautifulSoup(new_product.content, "html.parser")

    d['title'].append(get_title(product_soup))
    d['price'].append(get_price(product_soup))
    d['rating'].append(get_rating(product_soup))
    d['reviews'].append(get_reviews(product_soup))
    d['monthly_sales'].append(get_sales(product_soup))
    d['URL'].append("https://www.amazon.sg" + one_link)

import pandas as pd
amazon_df = pd.DataFrame.from_dict(d)

amazon_df

,title,price,rating,reviews,monthly_sales,URL
0,CeraVe 100% Mineral Sunscreen SPF 50 | Face Su...,N/A,4.3,"76,598",N/A,https://www.amazon.sg/CeraVe-Sunscreen-Sensiti...
1,Aveeno Kids Continuous Protection Zinc Oxide M...,18.4,4.6,"1,393",50+ bought in past month,https://www.amazon.sg/Aveeno-Continuous-Protec...
2,Aveeno Protect + Hydrate Moisturizing Body Sun...,18.67,4.5,"9,275",50+ bought in past month,https://www.amazon.sg/Aveeno-Moisturizing-Weig...
3,BLUE LIZARD Sensitive Mineral Sunscreen with Z...,21.27,4.7,"15,962",N/A,https://www.amazon.sg/BLUE-LIZARD-Sensitive-Pr...
4,CeraVe Anti Aging Face Cream with SPF 30 Sunsc...,N/A,4.3,"8,985",N/A,https://www.amazon.sg/CeraVe-Sunscreen-Wrinkle...
5,Banana Boat Sport Ultra SPF 50 Sunscreen Lotio...,18.85,4.6,"12,565",50+ bought in past month,https://www.amazon.sg/Banana-Sunscreen-Lotion-...
6,Aveeno Kids Continuous Protection Zinc Oxide M...,19.21,4.4,"8,754",N/A,https://www.amazon.sg/Aveeno-Continuous-Protec...
7,Aveeno Protect + Hydrate Sunscreen Moisturizin...,46.83,4.6,"3,756",N/A,https://www.amazon.sg/Aveeno-Sunscreen-Moistur...
8,Neutrogena Ultra Sheer Moisturizing Face Serum...,18.79,4.6,"6,055",N/A,https://www.amazon.sg/Neutrogena-Moisturizing-...
9,CeraVe 100% Mineral Sunscreen SPF 50 Body Suns...,22.16,4.0,"4,416",N/A,https://www.amazon.sg/CeraVe-Sunscreen-Sensiti...


In [ ]:
import csv

with open('AmazonWebScraper_SunscreenSingapore.csv', 'w', newline='', encoding='UTF8') as f:
    amazon_df.to_csv('AmazonWebScraper_SunscreenSingapore.csv', index=False)

In [ ]:
#Appending data to the csv

with open('AmazonWebScraper_SunscreenSingapore.csv', 'a+', newline='', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(data)